In [3]:
import sqlite3
conn = sqlite3.connect('nominations.db')

In [4]:
schema=conn.execute('pragma table_info(nominations);').fetchall()

In [5]:
first_ten=conn.execute('select * from nominations limit 10;').fetchall()

In [6]:
for elem in first_ten:
    print(elem)

(1, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', '0', 1)
(2, 'Actor -- Leading Role', 'Jeff Bridges', 'True Grit', 'Rooster Cogburn', '0', 1)
(3, 'Actor -- Leading Role', 'Jesse Eisenberg', 'The Social Network', 'Mark Zuckerberg', '0', 1)
(4, 'Actor -- Leading Role', 'Colin Firth', "The King's Speech", 'King George VI', '1', 1)
(5, 'Actor -- Leading Role', 'James Franco', '127 Hours', 'Aron Ralston', '0', 1)
(6, 'Actor -- Supporting Role', 'Christian Bale', 'The Fighter', 'Dicky Eklund', '1', 1)
(7, 'Actor -- Supporting Role', 'John Hawkes', "Winter's Bone", 'Teardrop', '0', 1)
(8, 'Actor -- Supporting Role', 'Jeremy Renner', 'The Town', 'James Coughlin', '0', 1)
(9, 'Actor -- Supporting Role', 'Mark Ruffalo', 'The Kids Are All Right', 'Paul', '0', 1)
(10, 'Actor -- Supporting Role', 'Geoffrey Rush', "The King's Speech", 'Lionel Logue', '0', 1)


In [7]:
sql = 'create table ceremonies (id integer primary key, Year integer, Host text)'
conn.execute(sql).fetchall()

OperationalError: table ceremonies already exists

In [ ]:
conn.execute('pragma table_info(ceremonies);').fetchall()

In [ ]:
years_hosts = [(2010, "Steve Martin"),
               (2009, "Hugh Jackman"),
               (2008, "Jon Stewart"),
               (2007, "Ellen DeGeneres"),
               (2006, "Jon Stewart"),
               (2005, "Chris Rock"),
               (2004, "Billy Crystal"),
               (2003, "Steve Martin"),
               (2002, "Whoopi Goldberg"),
               (2001, "Steve Martin"),
               (2000, "Billy Crystal"),
            ]
insert_query = "INSERT INTO ceremonies (Year, Host) VALUES (?,?);"

In [ ]:
conn.executemany(insert_query, years_hosts)

In [ ]:
#sql = 'delete from ceremonies'
#conn.execute(sql).fetchall()
sql = 'select * from ceremonies'
conn.execute(sql).fetchall()

#### Foreign Key Constraints

In [ ]:
sql = 'PRAGMA foreign_keys = ON'
conn.execute(sql).fetchall()

#### Setting OneToMany relation

In [ ]:
sql = 'create table nominations_two(id integer primary key, category text, nominee text, movie text, character text, won text, ceremony_id, FOREIGN KEY(ceremony_id) REFERENCES ceremonies(id))'
conn.execute(sql).fetchall()

In [ ]:
conn.execute('pragma table_info(nominations_two);').fetchall()

In [ ]:
sql='SELECT nominations.category, nominations.nominee, nominations.movie, nominations.character, nominations.won, ceremonies.id FROM nominations INNER JOIN ceremonies ON nominations.year == ceremonies.year;'
joined_nominations = conn.execute(sql).fetchall()

##### insert into new table with foreign keys

In [ ]:
insert_query2 = "INSERT INTO nominations_two (category, nominee, movie, character, won, ceremony_id) VALUES (?,?,?,?,?,?);"
conn.executemany(insert_query2, joined_nominations)

In [8]:
sql = 'select * from nominations_two limit 5'
conn.execute(sql).fetchall()

OperationalError: no such table: nominations_two

#### Deleting And Renaming Tables

In [ ]:
sql = 'drop table nominations'
conn.execute(sql).fetchall()

In [ ]:
sql = 'alter table nominations_two rename to nominations'
conn.execute(sql).fetchall()

#### Creating A Join Table

In [ ]:
sql = 'create table movies(id integer primary key, movie text)'
conn.execute(sql).fetchall()
sql = 'create table actors(id integer primary key, actor text)'
conn.execute(sql).fetchall()

In [ ]:
sql = 'create table movies_actors(id integer primary key, movie_id integer, actor_id integer, foreign key(movie_id) references movies(id), foreign key(actor_id) references actors(id))'
conn.execute(sql).fetchall()

#### Fill up join tables

In [ ]:
sql = 'select distinct movie from nominations'
movies_list = conn.execute(sql).fetchall()
sql = 'select distinct nominee from nominations'
actors_list = conn.execute(sql).fetchall()

In [ ]:
insert_movies = "INSERT INTO movies (movie) VALUES (?);"
conn.executemany(insert_movies, movies_list)
sql = 'select * from movies'
conn.execute(sql).fetchall()

In [ ]:
insert_actors = "INSERT INTO actors (actor) VALUES (?);"
conn.executemany(insert_actors, actors_list)
sql = 'select * from actors'
conn.execute(sql).fetchall()

In [ ]:
sql='SELECT movies.id, actors.id FROM nominations INNER JOIN actors ON actors.actor == nominations.nominee INNER JOIN movies ON movies.movie == nominations.movie'
joined_movies_actors = conn.execute(sql).fetchall()

In [ ]:
insert_movies_actors = "INSERT INTO movies_actors (movie_id, actor_id) VALUES (?,?);"
conn.executemany(insert_movies_actors, joined_movies_actors)

In [ ]:
sql='SELECT * FROM movies_actors'
conn.execute(sql).fetchall()

#### Add movie_actor id into nominations table

In [ ]:
sql = 'create table nominations_three(id integer primary key, category text, character text, won text, ceremony_id int, movie_actor_id integer, FOREIGN KEY(ceremony_id) REFERENCES ceremonies(id), FOREIGN KEY(movie_actor_id) REFERENCES movie_actors(id))'
conn.execute(sql).fetchall()

In [ ]:
sql = 'PRAGMA table_info(nominations_three)'
conn.execute(sql).fetchall()

In [10]:
sql = 'select category, character, won, ceremony_id, movies_actors.id from nominations inner join actors on actors.actor == nominations.nominee inner join movies on movies.movie == nominations.movie inner join movies_actors on movies_actors.movie_id == movies.id and movies_actors.actor_id == actors.id'
joined_movies_actors = conn.execute(sql).fetchall()

In [11]:
insert_movies_actors = "INSERT INTO nominations_three (category, character, won, ceremony_id, movie_actor_id) VALUES (?,?,?,?,?);"
conn.executemany(insert_movies_actors, joined_movies_actors)

In [16]:
sql = 'drop table nominations'
conn.execute(sql).fetchall()
sql = 'alter table nominations_three rename to nominations'
conn.execute(sql).fetchall()

OperationalError: no such table: nominations